### Import Packages and Prams

In [1]:
import pandas as pd
from py2neo import Graph, Node, Relationship, NodeMatcher
from neo4j import GraphDatabase
import matplotlib
import matplotlib.pyplot as plt
import yaml

In [3]:


# Load the configuration file
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

# Accessing keys and credentials
# openai_api_key = config["openai"]["api_key"]
neo4j_uri = config["neo4j"]["uri"]
neo4j_user = config["neo4j"]["user"]
neo4j_password = config["neo4j"]["password"]

## Establishing Connection

In [5]:

### Neo4j Connection Details
NEO4J_URI = neo4j_uri
NEO4J_USER = neo4j_user
NEO4J_PASSWORD = neo4j_password

# Establish connection
graph = Graph(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

### Checking the connection by looking over the data in the Database
# graph.run("MATCH (n) return count(n)")


In [17]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [18]:
conn = Neo4jConnection(uri=NEO4J_URI, user=NEO4J_USER, pwd=NEO4J_PASSWORD)

### Graph Schema

In [24]:
query = "CALL db.schema.visualization()"
conn.query(query)

[<Record nodes=[<Node element_id='-115' labels=frozenset({'ARTICLE'}) properties={'name': 'ARTICLE', 'indexes': [], 'constraints': []}>, <Node element_id='-117' labels=frozenset({'COLOR'}) properties={'name': 'COLOR', 'indexes': [], 'constraints': []}>, <Node element_id='-116' labels=frozenset({'DEPT'}) properties={'name': 'DEPT', 'indexes': [], 'constraints': []}>, <Node element_id='-118' labels=frozenset({'INDEX'}) properties={'name': 'INDEX', 'indexes': [], 'constraints': []}>, <Node element_id='-119' labels=frozenset({'GARMENT'}) properties={'name': 'GARMENT', 'indexes': [], 'constraints': []}>] relationships=[<Relationship element_id='-114' nodes=(<Node element_id='-115' labels=frozenset({'ARTICLE'}) properties={'name': 'ARTICLE', 'indexes': [], 'constraints': []}>, <Node element_id='-118' labels=frozenset({'INDEX'}) properties={'name': 'INDEX', 'indexes': [], 'constraints': []}>) type='GROUP_OF' properties={'name': 'GROUP_OF'}>, <Relationship element_id='-113' nodes=(<Node elemen

#### Count(NODES)

In [23]:
query = """MATCH (n) RETURN labels(n)[0] AS Label, COUNT(n) AS Count ORDER BY Count DESC"""
conn.query(query)


[<Record Label='ARTICLE' Count=45875>,
 <Record Label='DEPT' Count=250>,
 <Record Label='COLOR' Count=50>,
 <Record Label='GARMENT' Count=21>,
 <Record Label='INDEX' Count=10>]

#### COUNT(RELATIONSHIPS)

In [22]:
query = """MATCH ()-[r]->() RETURN TYPE(r) AS RelationshipType, COUNT(r) AS Count ORDER BY Count DESC"""
conn.query(query)

[<Record RelationshipType='COLOURED_IN' Count=90918>,
 <Record RelationshipType='DEPARTMENT_OF' Count=47244>,
 <Record RelationshipType='TYPE_OF' Count=46583>,
 <Record RelationshipType='GROUP_OF' Count=46351>]

#### COMMON(PRODUCTGROUPS) ---> Most Common ProductGroups

In [20]:
query = """MATCH (p:ARTICLE) RETURN p.product_group_name AS ProductGroup, COUNT(*) AS Count ORDER BY Count DESC LIMIT 10"""
conn.query(query)

[<Record ProductGroup='Garment Upper body' Count=16784>,
 <Record ProductGroup='Garment Lower body' Count=8396>,
 <Record ProductGroup='Garment Full body' Count=6644>,
 <Record ProductGroup='Accessories' Count=6180>,
 <Record ProductGroup='Shoes' Count=2630>,
 <Record ProductGroup='Underwear' Count=2189>,
 <Record ProductGroup='Swimwear' Count=1385>,
 <Record ProductGroup='Socks & Tights' Count=788>,
 <Record ProductGroup='Nightwear' Count=737>,
 <Record ProductGroup='Unknown' Count=50>]

#### COMMON(COLOR)  --->  Most Common Color

In [26]:
query = """MATCH (c:COLOR) RETURN c.name AS Color, COUNT(*) AS Count ORDER BY Count DESC LIMIT 10 """
conn.query(query)

[<Record Color='Other Green' Count=1>,
 <Record Color='Other Blue' Count=1>,
 <Record Color='Light Purple' Count=1>,
 <Record Color='Dark Turquoise' Count=1>,
 <Record Color='Other Purple' Count=1>,
 <Record Color='Bronze/Copper' Count=1>,
 <Record Color='Black' Count=1>,
 <Record Color='White' Count=1>,
 <Record Color='Off White' Count=1>,
 <Record Color='Other Turquoise' Count=1>]

### Top Connected Nodes

In [28]:
query = """MATCH (n)-[r]-()
RETURN labels(n)[0] AS NodeLabel, n.name AS NodeName, COUNT(r) AS Degree
ORDER BY Degree DESC
LIMIT 10
"""

conn.query(query)

[<Record NodeLabel='ARTICLE' NodeName=None Degree=231096>,
 <Record NodeLabel='COLOR' NodeName='Black' Degree=18775>,
 <Record NodeLabel='INDEX' NodeName='Ladieswear' Degree=12944>,
 <Record NodeLabel='COLOR' NodeName='Dark Blue' Degree=9244>,
 <Record NodeLabel='GARMENT' NodeName='Jersey Fancy' Degree=8140>,
 <Record NodeLabel='COLOR' NodeName='White' Degree=7686>,
 <Record NodeLabel='INDEX' NodeName='Divided' Degree=6786>,
 <Record NodeLabel='GARMENT' NodeName='Accessories' Degree=6272>,
 <Record NodeLabel='COLOR' NodeName='Light Pink' Degree=4848>,
 <Record NodeLabel='INDEX' NodeName='Menswear' Degree=4624>]

### Frequent Colors in Articles

In [29]:
query = """MATCH (a:ARTICLE)-[:COLOURED_IN]->(c:COLOR)
RETURN c.name AS Color, COUNT(a) AS ArticleCount
ORDER BY ArticleCount DESC
LIMIT 10"""

conn.query(query)

[<Record Color='Black' ArticleCount=18775>,
 <Record Color='Dark Blue' ArticleCount=9244>,
 <Record Color='White' ArticleCount=7686>,
 <Record Color='Light Pink' ArticleCount=4848>,
 <Record Color='Grey' ArticleCount=3783>,
 <Record Color='Light Beige' ArticleCount=3112>,
 <Record Color='Red' ArticleCount=2759>,
 <Record Color='Blue' ArticleCount=2725>,
 <Record Color='Light Blue' ArticleCount=2616>,
 <Record Color='Beige' ArticleCount=2523>]

### Relationship Between Garments and Index

In [30]:
query = """MATCH (a:ARTICLE)-[:TYPE_OF]->(g:GARMENT), (a)-[:GROUP_OF]->(i:INDEX)
RETURN g.name AS GarmentType, i.name AS IndexName, COUNT(a) AS ArticleCount
ORDER BY ArticleCount DESC
LIMIT 10
"""

conn.query(query)

[<Record GarmentType='Jersey Fancy' IndexName='Ladieswear' ArticleCount=2953>,
 <Record GarmentType='Accessories' IndexName='Ladies Accessories' ArticleCount=2940>,
 <Record GarmentType='Blouses' IndexName='Ladieswear' ArticleCount=2340>,
 <Record GarmentType='Under-, Nightwear' IndexName='Lingeries/Tights' ArticleCount=1933>,
 <Record GarmentType='Trousers' IndexName='Ladieswear' ArticleCount=1427>,
 <Record GarmentType='Dresses Ladies' IndexName='Ladieswear' ArticleCount=1329>,
 <Record GarmentType='Knitwear' IndexName='Ladieswear' ArticleCount=1308>,
 <Record GarmentType='Accessories' IndexName='Children Accessories, Swimwear' ArticleCount=1290>,
 <Record GarmentType='Unknown' IndexName='Ladieswear' ArticleCount=1270>,
 <Record GarmentType='Jersey Fancy' IndexName='Baby Sizes 50-98' ArticleCount=1166>]

### Find Popular Departments

In [31]:
query = """MATCH (a:ARTICLE)-[:DEPARTMENT_OF]->(d:DEPT)
RETURN d.name AS Department, COUNT(a) AS ArticleCount
ORDER BY ArticleCount DESC
LIMIT 10
"""

conn.query(query)

[<Record Department='Jersey' ArticleCount=2285>,
 <Record Department='Knitwear' ArticleCount=1460>,
 <Record Department='Trouser' ArticleCount=1110>,
 <Record Department='Dress' ArticleCount=974>,
 <Record Department='Blouse' ArticleCount=943>,
 <Record Department='Swimwear' ArticleCount=875>,
 <Record Department='Expressive Lingerie' ArticleCount=853>,
 <Record Department='Jewellery' ArticleCount=807>,
 <Record Department='Outwear' ArticleCount=634>,
 <Record Department='Blouse & Dress' ArticleCount=604>]

### Findinng Similar Articles 

In [35]:
query = """MATCH (a1:ARTICLE)-[:DEPARTMENT_OF]->(d:DEPT),
      (a2:ARTICLE)-[:DEPARTMENT_OF]->(d)
WHERE a1 <> a2 AND a1.product_group_name = a2.product_group_name
RETURN a1.title AS Article1, a2.title AS Article2, d.name AS Department, a1.product_group_name AS ProductGroup limit 100;"""

print(conn.query(query))

[<Record Article1='V-NECK SS BASIC 3 PK' Article2='Frank Tank' Department='Light Basic Jersey' ProductGroup='Garment Upper body'>, <Record Article1='Taz Tee' Article2='Frank Tank' Department='Light Basic Jersey' ProductGroup='Garment Upper body'>, <Record Article1='R-NECK SS BASIC FIT' Article2='Frank Tank' Department='Light Basic Jersey' ProductGroup='Garment Upper body'>, <Record Article1='R-NECK SS SLIM FIT' Article2='Frank Tank' Department='Light Basic Jersey' ProductGroup='Garment Upper body'>, <Record Article1='V-NECK SS SLIM FIT' Article2='Frank Tank' Department='Light Basic Jersey' ProductGroup='Garment Upper body'>, <Record Article1='R-NECK LS SLIM FIT' Article2='Frank Tank' Department='Light Basic Jersey' ProductGroup='Garment Upper body'>, <Record Article1='VICTOR 3PK SLIM VN T-SHIRT' Article2='Frank Tank' Department='Light Basic Jersey' ProductGroup='Garment Upper body'>, <Record Article1='V-NECK SS SLIM 3 PK' Article2='Frank Tank' Department='Light Basic Jersey' ProductGro